# 1. Testing different Models

In [2]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel

dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

model = ExplicitFactorizationModel(n_iter=1)
model.fit(train)

rmse = rmse_score(model, test)
rmse

0.99168646

Tareas:
1. Mirar y resumir los dsitintos modelos, probarlos
2. Probar distintos tamaños de los modelos
3. Hacer el 

In [3]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

model = ImplicitFactorizationModel(n_iter=3,
                                   loss='bpr')
model.fit(train)

mrr = mrr_score(model, test)

# 2. Testing new user film imputs

In [15]:
len(np.unique(dataset.user_ids))

943

In [10]:
dataset.item_ids

array([ 242,  302,  377, ..., 1090,  225,  203], dtype=int32)

In [9]:
dataset.ratings

array([3., 3., 1., ..., 1., 2., 3.], dtype=float32)

In [4]:
import numpy as np
from spotlight.interactions import Interactions

us = [93,944]
its = [23,12]
us = np.array(us, dtype=np.int32)
its = np.array(its, dtype=np.int32)
rate = [3,4]
rate = np.array(rate, dtype=np.float32)

new = Interactions(user_ids=us, item_ids=its, ratings=rate)
model.fit(new)

AttributeError: 'Interactions' object has no attribute 'sequences'

It appears thar for every new user or film the model needs to be scratched and retraiened from 0 for factoricer models. We will then proceed with sequential models.

In [ ]:
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.synthetic import generate_sequential
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.datasets.movielens import get_movielens_dataset


dataset = get_movielens_dataset(variant='20M')


train, test = user_based_train_test_split(dataset)

train = train.to_sequence()
test = test.to_sequence()

model = ImplicitSequenceModel(n_iter=3,
                              representation='cnn',
                              loss='bpr')
model.fit(train)

mrr = sequence_mrr_score(model, test)
mrr

In [ ]:
import numpy as np
its = [23,12,22,34,42,55,60,20,1,2,12]
its = np.array(its, dtype=np.int32)

pred = model.predict(its)
pred

# New users need intermidiate functions for interpretation

In [59]:
import pandas as pd
import difflib

def get_movie_identifier(item_id):
    """
    Gets the movie imdbId and tmdbId for a movie title
    """
    links = pd.read_csv('links.csv')
    return links['imdbId'][links['movieId']==item_id].values[0]

def get_movie_id(title):
    """
    Gets the movie id for a movie title match
    """
    movies = pd.read_csv('movies.csv')
    

    existing_titles = list(movies['title'].values)
    closest_titles = difflib.get_close_matches(title, existing_titles)
    
    if len(closest_titles)==0: 
        print("Spelling Mistake in titles \n or not in library")
        return 1
    movie_id =  movies['movieId'][movies['title']==closest_titles[0]].values[0]
    return movie_id


def get_movie_title(movie_id):
    """
    Gets the movie imdbId and tmdbId for a movie title
    """
    movies = pd.read_csv('movies.csv')
    return movies['title'][movies['movieId']==movie_id].values[0]

In [47]:
get_movie_identifier(1)

array([114709])

In [48]:
get_movie_title(2)

'Jumanji (1995)'

In [23]:
torch.load()

NameError: name 'torch' is not defined

# Function for movie recommendations for new users.

In [62]:
import torch

def recommend_new_sequence(movies, model_path, n_recom=5):
    """
    Receives a trained model path, a sequence of movie titles,
    associates them to a movielens ID
    and then predicst n_recoms new movies
    for the user and returns their titles.
    """
    movies_ids = [get_movie_id(m) for m in movies]
    spotlight_model = torch.load(model_path) # Load model
    
    pred = spotlight_model.predict(movies_ids) # In the prediction we could specify a specific subpool in future filter implemtation
    indices = np.argpartition(pred, -n_recom)[-n_recom:] 
    best_movie_ids = indices[np.argsort(pred[indices])]
    titles = [get_movie_title(b+1) for b in best_movie_ids]
    return titles

In [63]:
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.synthetic import generate_sequential
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.datasets.movielens import get_movielens_dataset
import torch
import numpy as np

dataset = get_movielens_dataset(variant='100K')
train = dataset.to_sequence()

model = ImplicitSequenceModel(n_iter=3,
                              representation='cnn',
                              loss='bpr')
model.fit(train)
model_path = "testmodel"
torch.save(model, model_path)

movies = np.array(['Sabrina', 'Mortal Kombat'])
recommend_new_sequence(movies, model_path)

['Guardian Angel (1994)',
 "Jupiter's Wife (1994)",
 'Only You (1994)',
 'Nina Takes a Lover (1994)',
 'Kiss of Death (1995)']